Плюс использования полиномиальных хэшей для строк - это быстрое вычисление хэша (за O(1)) для любой подстроки исхходной строки, если предпосчитать хэши для всех префиксов строки. Будем делать это, как описано в статье https://habrahabr.ru/post/142589/

In [1]:
modulo = pow(2, 32)


def FindAllSubstringsInReference(reference_string, substrings):
    """
    Find all entrances of given substrings into reference_string
    
    params: 
    reference_string (str): reference srting
    substrings {dict}: dict of substrings {number of substring: substring}
    """
    p = 401  # prime numper, bigger than biggest code of symbol in string
    # ord('t') = 116
    
    powers = [pow(p,i) for i in range(len(reference_string))]

    reference_hash = [0] * len(reference_string)
    reference_hash[0] = ord(reference_string[0]) % modulo
    for index in range(1, len(reference_string)):
        reference_hash[index] = (reference_hash[index - 1] + 
                                 powers[index] * ord(reference_string[index])) % modulo
    
    result = {}
    for index in substrings.keys():
        result[index] = FindSubstringEntrances(reference_hash, substrings[index], 
                                   powers, reference_string)
        print("Substring number {}: {}".format(index, substrings[index]))
        print("Positions of entrence (first letter) into reference string: ", ", ".join(list(map(str, result[index]))))


def GetHashOfSubstring(prefix_hashes, L, R):
    """
    Calculate hash of substring reference_string[L:R] using hashes of reference_string prefixes 
    """
    if L == 0:
        return prefix_hashes[R]
    return (prefix_hashes[R] - prefix_hashes[L - 1]) % modulo


def FindSubstringEntrances(ref_hash, substring, powers, reference_string):
    """
    Find all positions of entrance of substring into reference_string
    """
    substring_hash = sum([(ord(substring[i]) * powers[i]) for i in range(len(substring))]) % modulo
    substring_len = len(substring)
    indices_of_entrance = []
    for index in range(0, len(ref_hash) - substring_len + 1):
        if (GetHashOfSubstring(ref_hash, index, index + substring_len - 1) == 
                                    substring_hash * powers[index] % modulo):
            if reference_string[index:index + substring_len] == substring:
                indices_of_entrance.append(index)
            else:
                print("collision happened")
    return indices_of_entrance


with open("test5.txt", 'r') as file:
    lines = file.readlines()
reference_string = lines[0].strip()
lines.pop(0)
substrings = dict()
for line in lines:
    line = line.strip()
    current_string_number = 0
    if line.startswith(">"):
        currrent_string_number = line[1:]
    else:
        substrings[currrent_string_number] = line

FindAllSubstringsInReference(reference_string, substrings)

Substring number 1: cat
Positions of entrence (first letter) into reference string:  9, 102, 126, 183, 189
Substring number 2: aag
Positions of entrence (first letter) into reference string:  17, 44, 59, 137, 162
